In [ ]:
import sys
sys.path.insert(0, './bitcoin-taproot/test/functional/')
srcdir = "./bitcoin-taproot/src/" 

from test_framework.test_framework import BitcoinTestFramework
from test_framework.key import *
import binascii
import hashlib
import random

# TODO: Add ExampleTest/setup_test/shutdown_test from utils.py

# Adaptor Signatures

![test](images/adaptor_intro_0.jpg)


TODO: Description


### 1)  Generate (Musig) Signatures

In [ ]:
num_participants = random.randint(3,6)

keys = []
pubkeys = []
for _ in range(num_participants):
    private_key = ECKey()
    private_key.generate()
    public_key = private_key.get_pubkey()
    keys.append((private_key, public_key))
    pubkeys.append(public_key)

c_map, pk_musig = generate_musig_key(pubkeys)

# Public Key & Challenge Factors.
keys_c = []
for private, public in keys:
    private_c = private.mul(c_map[public])
    public_c = public.mul(c_map[public])
    keys_c.append((private_c, public_c))

# Nonce.
nonce_map = {}
nonce_points = []
for private_c, public_c in keys_c:
    nonce_map[public_c] = generate_schnorr_nonce()
    nonce_points.append(nonce_map[public_c].get_pubkey())

R_agg, negated = aggregate_schnorr_nonces(nonce_points)

# Indivdiual Signatures.
msg = hashlib.sha256(b'transaction').digest()
sigs = []
for private_c, public_c in keys_c:
    signature = private_c.sign_musig(nonce_map[public_c], negated, R_agg, pk_musig, msg)
    sigs.append(signature)
    

### 2) Adaptor Secret and Point.

In [ ]:
t = hashlib.sha256(b'adaptor tweak').digest()
tk = ECKey()
tk.set(t, True)
Tk = tk.get_pubkey()


### 3) Tweak a Signature to Create an Adaptor Sig

In [ ]:
# Tweak a signature.
i = random.randint(0, len(sigs)-1)
sig = sigs[i]
sig_adaptor = tweak_signature(sig, t)
    

### 4) Counterparty Verifies Adaptor Signature

In [ ]:
private_c, public_c = keys_c[i]
print(public_c.verify_adaptor(sig_adaptor, Tk, R_agg, pk_musig, msg))


### 5) Counterparty recoveres Secret from Adaptor and Valid Signature.

In [ ]:
# Recover Secret
t_recovered = get_adaptor_tweak(sig_adaptor, sig)
print(t_recovered == t)


### 6) Convert an adaptor to a valid signature with adaptor secret t.

In [ ]:
# R, s' -> R, s
sig_recovered = tweak_signature(sig_adaptor,t,negate=True)
print(sig_recovered == sig)
